In [108]:
import pandas as pd

In [109]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)

In [110]:
transactions

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
0,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,О
1,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,И
2,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,ЗО
3,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,И
4,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,О
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2023-06-23 10:33:59,2023-06-27,МТС-ао,MTSS,RUB,Покупка,10,343.350,3433.5,0.0,2.06,1.03,7792729150,О
263,2023-07-18 16:07:57,2023-07-20,СевСт-ао,CHMF,RUB,Покупка,1,1275.600,1275.6,0.0,0.77,0.38,7908972071,И
264,2023-07-18 17:52:50,2023-07-20,Сбербанк,SBER,RUB,Покупка,10,246.210,2462.1,0.0,1.48,0.73,7909781928,И
265,2023-07-18 17:55:20,2023-07-20,Системаао,AFKS,RUB,Покупка,100,17.744,1774.4,0.0,1.06,0.54,7909796018,И


In [111]:
transactions_executed = transactions[transactions['Статус'] == 'И']

In [112]:
transactions_executed

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
1,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,И
3,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,И
6,2021-05-17 10:28:00,2021-05-19,FXRLETF,FXRL,RUB,Покупка,1,3896.500,3896.5,0.0,2.34,0.36,3943169446,И
9,2021-05-17 10:32:49,2021-05-19,FXDMETF,FXDM,RUB,Покупка,40,75.970,3038.8,0.0,1.82,0.28,3943195531,И
12,2021-05-17 10:38:09,2021-05-19,FXRUETF,FXRU,RUB,Покупка,2,951.400,1902.8,0.0,1.14,0.18,3943220803,И
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,2023-06-23 10:33:59,2023-06-27,МТС-ао,MTSS,RUB,Покупка,10,343.350,3433.5,0.0,2.06,1.03,7792729150,И
263,2023-07-18 16:07:57,2023-07-20,СевСт-ао,CHMF,RUB,Покупка,1,1275.600,1275.6,0.0,0.77,0.38,7908972071,И
264,2023-07-18 17:52:50,2023-07-20,Сбербанк,SBER,RUB,Покупка,10,246.210,2462.1,0.0,1.48,0.73,7909781928,И
265,2023-07-18 17:55:20,2023-07-20,Системаао,AFKS,RUB,Покупка,100,17.744,1774.4,0.0,1.06,0.54,7909796018,И


In [113]:
transactions_executed.loc[(transactions_executed['Код'] == 'SBMX') & (transactions_executed['Цена'] > 100), 'Количество'] = \
transactions_executed.loc[(transactions_executed['Код'] == 'SBMX') & (transactions_executed['Цена'] > 100), 'Количество'] * 100

In [119]:
code_list = transactions_executed['Код'].unique()
share_price_dict = {}
for code in code_list:
    if code in ['SBMX', 'AFKS']:
        round_numb = 3
    elif code in ['IRAO']:
        round_numb = 4
    else:
        round_numb = 2
    share = transactions_executed[transactions_executed['Код'] == code]
    share_price = ((share['Сумма'].sum() + share['Комиссия Брокера'].sum() + share['Комиссия Биржи'].sum()) / 
                        (share.loc[share['Вид'] == 'Покупка', 'Количество'].sum() - share.loc[share['Вид'] == 'Продажа', 'Количество'].sum())).round(round_numb)
    share_price_dict[code] = share_price

In [120]:
share_price_dict

{'SBSP': 1724.63,
 'FXUS': 377.52,
 'FXRL': 82.21,
 'FXDM': 76.64,
 'FXRU': 253.43,
 'FXDE': 153.14,
 'FXIM': 90.12,
 'SBMX': 15.757,
 'FXRW': 1.28,
 'VTBE': 93.89,
 'FXGD': 1838.48,
 'RU000A101FA1': 987.42,
 'SBER': 224.66,
 'MTSS': 300.9,
 'GAZP': 174.17,
 'LKOH': 4821.33,
 'MGNT': 4186.26,
 'YNDX': 2642.38,
 'CHMF': 1276.75,
 'AFKS': 17.76,
 'IRAO': 4.1628}

In [116]:
transactions_executed[transactions_executed['Код'] == 'FXUS']

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
3,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.00,5615.00,0.0,3.37,0.52,3943142919,И
38,2021-05-25 18:28:44,2021-05-27,FXUSETF,FXUS,RUB,Покупка,1,5644.00,5644.00,0.0,3.39,0.52,3970365261,И
44,2021-06-11 22:19:00,2021-06-15,FXUSETF,FXUS,RUB,Покупка,1,5616.00,5616.00,0.0,3.37,0.52,4029471218,И
162,2022-01-24 13:01:49,2022-01-26,FXUSETF,FXUS,RUB,Покупка,45,62.46,2810.70,0.0,1.69,0.26,4903477636,И
183,2022-02-17 11:53:33,2022-02-21,FXUSETF,FXUS,RUB,Покупка,5,61.75,308.75,0.0,0.19,0.03,5002275241,И
